<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Ensemble_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Print the correct tensorflow version 

In [1]:
from google.colab import drive
try:
  drive.mount('/content/drive/',force_remount = True)
  COLAB = True
  print("Google colab is on")
  %tensorflow_version 2.x
except:
  COLAB = False
  print("Not using Google Colab")

Mounted at /content/drive/
Google colab is on


In [135]:
#Time formatter
def time_fmt(secs):
  hours = int(secs/(60*60))
  minutes = int(secs%(60*60)/60)
  seconds = int(secs%60)
  return f"{hours}:{minutes:>02}:{seconds:>05.2f}"

In [3]:
#Pertubation implementation to variable importance for NNT

In [136]:
import scipy as sc
import numpy as np
from sklearn import metrics
import math
import pandas as pd


In [137]:
#This function will compute the loss for a regression and classification model
#To provide usefull information in computing variable's importance
#First we select all columns from a test dataset
#Second we randomly shuffles every columns to distort any parttern
#If the fitted model is a regression we compute MSE otherwise we compute log-loss
#We compute variable importance by take the ratio of individual error vs maximum error
#We prepare a results dictionary which is finaly conveted to a dataframe.

In [161]:
def var_rank(model, x, y, names, regression):
  errors = []
  for i in range(x.shape[1]):
    hold = np.array(x[:,i])
    np.random.shuffle(x[:,i])
    if regression:
      pred = model.predict(x)
      err = metrics.mean_squared_error(y,pred)
    else:
      pred = model.predict_proba(x)
      err = metrics.log_loss(y, pred)
    errors.append(err)
    x[:,i] = hold
  e_max = np.max(errors) 
  importance = [e/e_max for e in errors]
  data = {'names': names,'errors': errors, 'importance': importance}
  out = pd.DataFrame(data, columns = ['names', 'errors', 'importance'])
  out.sort_values(by = ['importance'], ascending = [0],inplace = True)
  out.reset_index(inplace = True, drop = True)
  return out




In [139]:
#Build a Keras Sequential model for classification using Iris dataset

In [140]:
#Loading the packages
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import pandas as pd


In [141]:
iris = pd.read_csv("https://data.heatonresearch.com/data/t81-558/iris.csv",
                   na_values = ['NA','?'])

In [142]:
display(iris.head(10))

,sepal_l,sepal_w,petal_l,petal_w,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [143]:
iris.isna().value_counts()

sepal_l  sepal_w  petal_l  petal_w  species
False    False    False    False    False      150
dtype: int64

In [108]:
dummies = pd.get_dummies(iris['species'])
y = dummies.values



In [144]:
x = iris.drop(columns = ['species'],axis = 1)

In [145]:
x.head()

,sepal_l,sepal_w,petal_l,petal_w
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [146]:
x = x.values
x[0:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [147]:
#Split the data into train and test sets

In [148]:
x_train, x_test, y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state = 42)

In [149]:
x_train[0:2]

array([[5. , 3.6, 1.4, 0.2],
       [5.2, 4.1, 1.5, 0.1]])

In [150]:
y_train[0:2]

array([[1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [151]:
#Building keras Sequential NNT for classifying Iris flowers
model = Sequential()
model.add(Dense(50, input_dim = x.shape[1], activation = 'relu',kernel_initializer = 'random_normal'))
model.add(Dense(25, activation = 'relu', kernel_initializer = 'random_normal'))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
#info = EarlyStopping(monitor = 'val_loss',min_delta = 1e-3,patience = 5,verbose = 1, restore_best_weights = True)
model.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 2, epochs = 100)

Epoch 1/100
4/4 - 0s - loss: 1.1338 - val_loss: 1.1061
Epoch 2/100
4/4 - 0s - loss: 1.0979 - val_loss: 1.0848
Epoch 3/100
4/4 - 0s - loss: 1.0865 - val_loss: 1.0755
Epoch 4/100
4/4 - 0s - loss: 1.0782 - val_loss: 1.0684
Epoch 5/100
4/4 - 0s - loss: 1.0708 - val_loss: 1.0595
Epoch 6/100
4/4 - 0s - loss: 1.0641 - val_loss: 1.0509
Epoch 7/100
4/4 - 0s - loss: 1.0572 - val_loss: 1.0416
Epoch 8/100
4/4 - 0s - loss: 1.0506 - val_loss: 1.0333
Epoch 9/100
4/4 - 0s - loss: 1.0419 - val_loss: 1.0220
Epoch 10/100
4/4 - 0s - loss: 1.0334 - val_loss: 1.0108
Epoch 11/100
4/4 - 0s - loss: 1.0240 - val_loss: 0.9985
Epoch 12/100
4/4 - 0s - loss: 1.0130 - val_loss: 0.9850
Epoch 13/100
4/4 - 0s - loss: 1.0027 - val_loss: 0.9699
Epoch 14/100
4/4 - 0s - loss: 0.9895 - val_loss: 0.9553
Epoch 15/100
4/4 - 0s - loss: 0.9759 - val_loss: 0.9386
Epoch 16/100
4/4 - 0s - loss: 0.9630 - val_loss: 0.9198
Epoch 17/100
4/4 - 0s - loss: 0.9479 - val_loss: 0.9021
Epoch 18/100
4/4 - 0s - loss: 0.9324 - val_loss: 0.8830
E

In [152]:
preds = model.predict(x_test)

In [153]:
preds = np.argmax(preds, axis = 1)

In [154]:
expected = np.argmax(y_test, axis = 1)

In [155]:
accuracy_score = metrics.accuracy_score(preds, expected)
accuracy_score

0.9736842105263158

In [156]:
#Get the variable importance from the above `NNT
names = list(iris.columns)

In [157]:
 names.remove('species')

In [162]:
from IPython.display import display, HTML


VarImp = var_rank(model, x_test,y_test,names, False)

In [163]:
VarImp

,names,errors,importance
0,petal_l,2.358871,1.000000
1,petal_w,0.829603,0.351695
2,sepal_l,0.156032,0.066147
3,sepal_w,0.119048,0.050468


In [206]:

#NNT variable importance for regression
cars = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", na_values = ['NA','?'])

In [207]:
display(cars.head(5))

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [208]:
cars.isna().sum()
cars['year'].unique()

array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])

In [209]:
#Data cleaning and preprocess
#Fill the missing values for horsepower by the median value
cars['horsepower'] = cars['horsepower'].fillna(cars['horsepower'].median())

In [210]:
#Create dummies for the categorical variables
clyd = pd.get_dummies(cars['cylinders'],prefix ='cylinder')
origind = pd.get_dummies(cars['origin'],prefix='origin')
yearsd = pd.get_dummies(cars['year'], prefix='year')

In [211]:
#Get the target variable
y = cars['mpg']

In [212]:
x = cars.drop(columns = ['cylinders','year','origin','name','mpg'], axis = 1)

In [213]:
display(x.head())

,displacement,horsepower,weight,acceleration
0,307.0,130.0,3504,12.0
1,350.0,165.0,3693,11.5
2,318.0,150.0,3436,11.0
3,304.0,150.0,3433,12.0
4,302.0,140.0,3449,10.5


In [214]:
#Standerdize the data into z-scores
cols = list(x.columns)

In [215]:
for col in cols:
  x[col] = zscore(x[col])

In [216]:
display(x.head())

,displacement,horsepower,weight,acceleration
0,1.090604,0.673118,0.630870,-1.295498
1,1.503514,1.589958,0.854333,-1.477038
2,1.196232,1.197027,0.550470,-1.658577
3,1.061796,1.197027,0.546923,-1.295498
4,1.042591,0.935072,0.565841,-1.840117


In [217]:
#add the dummies columns to our final predictors
x = pd.concat([x,clyd, yearsd, origind], axis = 1)

In [218]:
display(x.head())

,displacement,horsepower,weight,acceleration,cylinder_3,cylinder_4,cylinder_5,cylinder_6,cylinder_8,year_70,year_71,year_72,year_73,year_74,year_75,year_76,year_77,year_78,year_79,year_80,year_81,year_82,origin_1,origin_2,origin_3
0,1.090604,0.673118,0.630870,-1.295498,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.503514,1.589958,0.854333,-1.477038,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1.196232,1.197027,0.550470,-1.658577,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1.061796,1.197027,0.546923,-1.295498,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1.042591,0.935072,0.565841,-1.840117,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [219]:
names1 = list(x.columns)

x = x.values
x[0:6]

array([[ 1.0906037 ,  0.67311762,  0.63086987, -1.29549834,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
       [ 1.5035143 ,  1.58995818,  0.85433297, -1.47703779,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ],
       [ 1.19623199,  1.19702651,  0.55047045, -1.65857724,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0

In [220]:
#standerdize the target
y = zscore(y)

In [221]:
y[0:6]

array([-0.7064387 , -1.09075062, -0.7064387 , -0.96264665, -0.83454267,
       -1.09075062])

In [222]:
#Spliting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 123)

In [223]:
#MLP NNT for regression
model = Sequential()
model.add(Dense(60, input_dim = x.shape[1], activation = 'relu', kernel_initializer = 'random_normal'))
model.add(Dense(30, activation = 'relu', kernel_initializer='random_normal'))
model.add(Dense(1))
info = EarlyStopping(monitor = 'val_loss', min_delta =1e-3, patience = 5, verbose = 1, restore_best_weights=True)
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(x_train, y_train, validation_data = (x_test, y_test), callbacks = [info], verbose = 2, epochs = 1000)


Epoch 1/1000
10/10 - 0s - loss: 0.9023 - val_loss: 0.7440
Epoch 2/1000
10/10 - 0s - loss: 0.6841 - val_loss: 0.5400
Epoch 3/1000
10/10 - 0s - loss: 0.4582 - val_loss: 0.3687
Epoch 4/1000
10/10 - 0s - loss: 0.3062 - val_loss: 0.3105
Epoch 5/1000
10/10 - 0s - loss: 0.2473 - val_loss: 0.2703
Epoch 6/1000
10/10 - 0s - loss: 0.2017 - val_loss: 0.2399
Epoch 7/1000
10/10 - 0s - loss: 0.1733 - val_loss: 0.2158
Epoch 8/1000
10/10 - 0s - loss: 0.1498 - val_loss: 0.1910
Epoch 9/1000
10/10 - 0s - loss: 0.1272 - val_loss: 0.1756
Epoch 10/1000
10/10 - 0s - loss: 0.1114 - val_loss: 0.1586
Epoch 11/1000
10/10 - 0s - loss: 0.1002 - val_loss: 0.1506
Epoch 12/1000
10/10 - 0s - loss: 0.0921 - val_loss: 0.1477
Epoch 13/1000
10/10 - 0s - loss: 0.0888 - val_loss: 0.1490
Epoch 14/1000
10/10 - 0s - loss: 0.0854 - val_loss: 0.1500
Epoch 15/1000
10/10 - 0s - loss: 0.0832 - val_loss: 0.1508
Epoch 16/1000
10/10 - 0s - loss: 0.0810 - val_loss: 0.1478
Epoch 17/1000
Restoring model weights from the end of the best ep

In [224]:
pred = model.predict(x_test)

In [225]:
mse = metrics.mean_squared_error(y_test, pred)
mse

0.1477069796815034

In [227]:
#Variable importance
Variables_ranks = var_rank(model,x_test,y_test,names1, True)

In [228]:
Variables_ranks

,names,errors,importance
0,weight,0.346180,1.000000
1,horsepower,0.304949,0.880896
2,displacement,0.226458,0.654162
3,year_72,0.187361,0.541225
4,year_82,0.183443,0.529907
5,year_73,0.177232,0.511964
6,year_80,0.171281,0.494775
7,year_75,0.170051,0.491221
8,year_74,0.167942,0.485130
9,year_71,0.161770,0.467300
